In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 14

In [ ]:
# select analysis to perform
CASE = 1
if CASE == 0: dname = 'DATA'
if CASE == 1: dname = 'DATA_b'
fname = dname + '/x_RBM_q0.1.dat'

# loading data: each row is a list of visible units
# NOTE: data "x" here is named "v" for "visible
v = np.loadtxt(fname, delimiter=' ', dtype=int)
# store in v0, because later we will shuffle v
v0 = np.copy(v)

# convert blocks of four bits to category index
for i in range(0, v0.shape[1], 4):
    v0[:, i] = np.argmax(v0[:, i:(i + 4)], axis=1)
# delete spurious columns and reverse indices to have
# 0001 --> 1
# 0010 --> 2
# 0100 --> 3
# 1000 --> 4
v0 = 4 - v0[:, range(0, v0.shape[1], 4)]